<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align=left>

# Vault Setup (dev mode)

**"Dev"** mode is a pre-configured server
* useful for local development, testing, and exploration
* not very secure
* in-memory
* Vault is automatically unsealed
* can optionally set the initial root token


Prereq: 
You need to have Vault binary


Command to execute (in separate windows):

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`


In [4]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [1]:
vault --version

Vault v1.6.0 (7ce0bd9691998e0443bc77e98b1e2a4ab1e965d4)


In [3]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.6.0
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-40c6ec7d
Cluster ID      88ceb9de-5d03-7f3b-d3a0-8d100c75923d
HA Enabled      true
HA Cluster      https://127.0.0.1:8201
HA Mode         active


#### Note the Initialized and Sealed status above.

## Interacting with Vault

Vault **HTTP API**
* Full access to Vault via HTTP
* Every aspect of Vault can be controlled via API


Vault **CLI**
* uses HTTP API to access Vault
* it is a thin wrapper around the HTTP API
* outputs are formatted

Example:  Getting help
`vault <command> -h`

In [4]:
vault -h

Usage: vault <command> [args]

Common commands:
    read        Read data and retrieves secrets
    write       Write data, configuration, and secrets
    delete      Delete secrets and configuration
    list        List data or secrets
    login       Authenticate locally
    agent       Start a Vault agent
    server      Start a Vault server
    status      Print seal and HA status
    unwrap      Unwrap a wrapped secret

Other commands:
    audit          Interact with audit devices
    auth           Interact with auth methods
    debug          Runs the debug command
    kv             Interact with Vault's Key-Value storage
    lease          Interact with leases
    monitor        Stream log messages from a Vault server
    namespace      Interact with namespaces
    operator       Perform operator-specific tasks
    path-help      Retrieve API help for paths
    plugin         Interact with Vault plugins and catalog
    policy         Interact with policies
    print          

In [5]:
vault server -h

Usage: vault server [options]

  This command starts a Vault server that responds to API requests. By default,
  Vault will start in a "sealed" state. The Vault cluster must be initialized
  before use, usually by the "vault operator init" command. Each Vault server must
  also be unsealed using the "vault operator unseal" command or the API before the
  server can respond to requests.

  Start a server with a configuration file:

      $ vault server -config=/etc/vault/config.hcl

  Run in "dev" mode:

      $ vault server -dev -dev-root-token-id="root"

  For a full list of examples, please see the documentation.

HTTP Options:

  -address=<string>
      Address of the Vault server. The default is https://127.0.0.1:8200. This
      can also be specified via the VAULT_ADDR environment variable.

  -agent-address=<string>
      Address of the Agent. This can also be specified via the
      VAULT_AGENT_ADDR environment variable.

  -ca-cert=<string>
      Path on the local disk to a sin

## Enable Audit Logging

Audit backend keeps a detailed log of all requests and responses to Vault.  Sensitive information is **obfuscated** by default (HMAC).

In [6]:
vault audit enable file file_path=/tmp/audit.log

Success! Enabled the file audit device at: file/


In [3]:
cat /tmp/audit.log |jq

{
  "time": "2020-12-24T07:24:23.030504Z",
  "type": "response",
  "auth": {
    "client_token": "hmac-sha256:43510274735afea704524fe636af99dfdd216b0ade274ae07eeb1044c6ccce32",
    "accessor": "hmac-sha256:29a3538aa4c60394043cb9065977e6b7095420f6986aede129ebd2cc223ef5de",
    "display_name": "token",
    "policies": [
      "root"
    ],
    "token_policies": [
      "root"
    ],
    "token_type": "service",
    "token_issue_time": "2020-12-24T15:23:11+08:00"
  },
  "request": {
    "id": "31d40fbb-9be4-99ab-eeba-519a539b6360",
    "operation": "update",
    "mount_type": "system",
    "client_token": "hmac-sha256:43510274735afea704524fe636af99dfdd216b0ade274ae07eeb1044c6ccce32",
    "client_token_accessor": "hmac-sha256:29a3538aa4c60394043cb9065977e6b7095420f6986aede129ebd2cc223ef5de",
    "namespace": {
      "id": "root"
    },
    "path": "sys/audit/file",
    "data": {
      "description": "hmac-sha256:37d76557af31a8b85a44e9f578593527d34a6a31e68541daa9fe3107362bc3d3",
      "loca

In [5]:
vault audit enable -path=file-raw file file_path=/tmp/audit-raw.log log_raw=true

Success! Enabled the file audit device at: file-raw/


In [6]:
cat /tmp/audit-raw.log |jq

{
  "time": "2020-12-24T07:26:33.060622Z",
  "type": "response",
  "auth": {
    "client_token": "root",
    "accessor": "fJ8e3QTuMKunEyab9ODjEA1M",
    "display_name": "token",
    "policies": [
      "root"
    ],
    "token_policies": [
      "root"
    ],
    "token_type": "service",
    "token_issue_time": "2020-12-24T15:23:11+08:00"
  },
  "request": {
    "id": "3cd1cf10-9fde-2390-897a-59a1fccc76c8",
    "operation": "update",
    "mount_type": "system",
    "client_token": "root",
    "client_token_accessor": "fJ8e3QTuMKunEyab9ODjEA1M",
    "namespace": {
      "id": "root"
    },
    "path": "sys/audit/file-raw",
    "data": {
      "description": "",
      "local": false,
      "options": {
        "file_path": "/tmp/audit-raw.log",
        "log_raw": "true"
      },
      "type": "file"
    },
    "remote_address": "127.0.0.1"
  },
  "response": {
    "mount_type": "system"
  }
}


## Generate a root token

Good Practice: The root tokens should be used only for just enough initial setup or in emergencies.  Use tokens with assigned set of policies based on your role in the organization.

When a situation arise and you need a root token, this task walks you through the steps to generate one

In [7]:
vault operator generate-root -generate-otp > otp.txt

In [12]:
pwd
cat nonce.txt

/Users/tio/mydemo/vault/vault_demos-jupyter
0a1d941a-fb30-7bc6-1057-f8b3f9fee295


In [11]:
vault operator generate-root -init -otp=$(cat otp.txt) -format=json | jq -r ".nonce" > nonce.txt

The **nonce** value (nonce.txt) should be distributed to all unseal key holders.  
Note:  The generation of a root token requires a quorem of unseal keys.

**Each unseal key holder** must execute the following command along with their unseal key

In [13]:
vault operator generate-root -nonce=$(cat nonce.txt) "anvKNuOKnLkTd5p4dQDfZ5Bv2b4We4tEIJXJ+divvVY="

Nonce            0a1d941a-fb30-7bc6-1057-f8b3f9fee295
Started          true
Progress         1/1
Complete         true
Encoded Token    FF8+HgUWAgtdPSQTYiAsUDksEFYKLRkSBks


The output should have **Encoded Token** which then need to be decoded

In [14]:
vault operator generate-root -decode="FF8+HgUWAgtdPSQTYiAsUDksEFYKLRkSBks" -otp=$(cat otp.txt)

s.SxNruZ2GrTRBi3qgg29gTPRx


In [16]:
vault token lookup s.SxNruZ2GrTRBi3qgg29gTPRx

Key                 Value
---                 -----
accessor            QBPktcYWq9FvkRcqkij6FLd7
creation_time       1608795880
creation_ttl        0s
display_name        root
entity_id           n/a
expire_time         <nil>
explicit_max_ttl    0s
id                  s.SxNruZ2GrTRBi3qgg29gTPRx
meta                <nil>
num_uses            0
orphan              true
path                auth/token/root
policies            [root]
ttl                 0s
type                service


### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

Only when you start the Vault server in development mode, you would see the key/value v2 secrets engine enabled at "secret/" path by default. However, when you run Vault in non-development mode, you would need to enable key/value secrets engine explicitly like other secrets engine.

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">